In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras as k
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import skimage.morphology as morp
from skimage.filters import rank


assert pd.__version__ == '0.23.4'
assert np.__version__ == '1.15.4'
assert k.__version__ == '2.1.3'

Using TensorFlow backend.


In [7]:
train = pd.read_csv("../data/train.csv")
train_labels = train["Label"].values
train_data = train.drop(labels = ["Label", "Img_Name"], axis = 1)

In [30]:
# Load all images. 

import os
import skimage.data
def load_data(data_dir):
    images = []
    
    file_names = [os.path.join(data_dir, f)
                  for f in os.listdir(data_dir) if f.endswith(".jpg")]
    
    for f in file_names:
        images.append(skimage.data.imread(f))
    
    return images

images = load_data('../data/2/Train')
print(len(images))

467
467


In [10]:
#explore shapes
for image in images[:3]:
    print('image.shape[before]', image.shape)

image.shape[before] (720, 1280, 3)
image.shape[before] (1080, 1920, 3)
image.shape[before] (720, 1280, 3)


In [11]:
#transform all images to 250x250
import skimage.transform
images250 = [skimage.transform.resize(image, (250, 250)) for image in images]

/Users/eelrufaie/Library/Python/3.6/lib/python/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/eelrufaie/Library/Python/3.6/lib/python/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [12]:
#confirm transformation
for image in images250[:3]:
    print('image.shape[after]', image.shape)

image.shape[after] (250, 250, 3)
image.shape[after] (250, 250, 3)
image.shape[after] (250, 250, 3)
467


In [26]:
#Load Labels 
data = pd.read_csv("../data/train.csv")
labels = data["Label"]

labels.head()

0    Speed Limit 80
1    Speed Limit 50
2    Speed Limit 50
3    Speed Limit 50
4    Speed Limit 50
Name: Label, dtype: object

In [32]:
#categorize labels..

import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

y_train_labels = np.array(labels)
X_train_data = np.array(images250)

num_categories = 6

## hack (FIXME).... train & label len do not match, so delete a row from labels.
y_train_labels = np.delete(y_train_labels, (15), axis=0)

#encode labels
label_encoder = LabelEncoder()
y_labels_encoded = label_encoder.fit_transform(y_train_labels)

y_labels_categorized = to_categorical(y_labels_encoded, num_categories)

print('len(X_train_data)', len(X_train_data))
print('len(y_train_labels)', len(y_train_labels))
print('len(y_labels_categorized)', len(y_labels_categorized))

468
467
len(X_train_data) 467
len(y_train_labels) 467
len(y_labels_categorized) 467


In [48]:
input_shape = (250, 250, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, data_format='channels_first'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

model.summary()

model.fit(X_train_data, y_labels_categorized, nb_epoch=10, batch_size=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 32, 248, 1)        72032     
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 30, 246, 32)       320       
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 28, 244, 32)       9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 122, 32)       0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 12, 120, 64)       18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 60, 64)         0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 6, 60, 64)         0         
__________